In [1]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime
# import hvplot.pandas
# import matplotlib.pyplot as plt
# from sklearn import svm
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from pandas.tseries.offsets import DateOffset
# from sklearn.metrics import classification_report

from utils.helpers import alpaca_symbol_data

# For creating the DB
database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

# prevent an ssl failure
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# There are 2 tables on the Wikipedia page
# we want the first table
first_table = payload[0]
second_table = payload[1]

df500table = first_table

sp500Tickers = df500table['Symbol'].values.tolist()
df500sectors = df500table.drop(columns={'Security','SEC filings','GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK','Founded'})
df500sectors.set_index('Symbol', inplace=True)
# print(sp500Tickers)

In [2]:
count = 0
tickers_0 = []
tickers_1 = []
tickers_2 = []
tickers_3 = []
tickers_4 = []
tickers_5 = []


size = len(sp500Tickers)
print(size)

for each_ticker in sp500Tickers:
    if count <= 99:
        tickers_0.append(each_ticker)
    if count >= 100 and count <= 199:
        tickers_1.append(each_ticker)
    if count >= 200 and count <= 299:
        tickers_2.append(each_ticker)
    if count >= 300 and count <= 399:
        tickers_3.append(each_ticker)
    if count >= 400 and count <= 499:
        tickers_4.append(each_ticker)
    if count >= 500:
        tickers_5.append(each_ticker)
    count = count + 1
# print(count)


print(len(tickers_0))
# print(tickers_0)
print(len(tickers_1))
# print(tickers_1)
print(len(tickers_2))
# print(tickers_2)
print(len(tickers_3))
# print(tickers_3)
print(len(tickers_4))
# print(tickers_4)
print(len(tickers_5))
# print(tickers_5)


504
100
100
100
100
100
4


In [3]:
# Establish a Baseline Performance
# Import the OHLCV dataset into a Pandas DataFrame
tickerList =[tickers_0, tickers_1, tickers_2, tickers_3, tickers_4, tickers_5]
# Set the ticker
tickers = tickers_0

# Set start datetimes n years from today
years_back = 1

# Get the symbol_data and store it into a variable ohlcv_df
# ohlcv_df = alpaca_symbol_data(tickers, years_back)
ohlcv_df = pd.DataFrame()

StockCount = 100
for tickers in tickerList:
    ohlcv_df = alpaca_symbol_data(tickers, years_back)
    display(ohlcv_df.head())
    ohlcv_df.to_sql('stocks_'+ str(StockCount), engine)
    StockCount +=100


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2019-04-12 04:00:00+00:00,35.00,35.2350,34.5000,34.69,4610524,30177,34.779432,AAL
2019-04-15 04:00:00+00:00,34.65,35.2400,33.6800,33.75,6390646,39076,34.047912,AAL
2019-04-16 04:00:00+00:00,33.96,34.0900,33.5900,34.03,4499901,31592,33.873974,AAL
2019-04-17 04:00:00+00:00,34.45,34.7200,34.1031,34.38,3772085,27478,34.455475,AAL
2019-04-18 04:00:00+00:00,34.43,34.6199,33.9000,34.37,3318655,20312,34.284227,AAL


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2019-04-12 04:00:00+00:00,29.73,30.01,29.63,29.75,7247791,29155,29.766689,CAG
2019-04-15 04:00:00+00:00,29.77,30.15,29.69,30.08,3480983,23174,29.978839,CAG
2019-04-16 04:00:00+00:00,30.16,30.44,29.86,30.09,3433109,21850,30.049244,CAG
2019-04-17 04:00:00+00:00,30.24,30.24,29.76,30.12,4629394,25463,30.072216,CAG
2019-04-18 04:00:00+00:00,30.08,30.60,30.01,30.52,4119437,23764,30.417022,CAG


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2019-04-12 04:00:00+00:00,80.15,80.74,80.01,80.52,908982,8093,80.412768,AJG
2019-04-15 04:00:00+00:00,80.50,80.73,80.12,80.61,681319,7487,80.551184,AJG
2019-04-16 04:00:00+00:00,80.75,81.07,80.63,80.90,669989,7748,80.834020,AJG
2019-04-17 04:00:00+00:00,81.05,81.05,79.25,80.26,1133917,11864,80.130978,AJG
2019-04-18 04:00:00+00:00,80.15,80.57,79.77,80.42,1030320,6188,80.332563,AJG


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2019-04-12 04:00:00+00:00,89.73,90.980,89.66,89.88,944642,9521,90.110812,DGX
2019-04-15 04:00:00+00:00,90.39,91.045,90.02,90.99,608142,7340,90.744688,DGX
2019-04-16 04:00:00+00:00,91.42,91.420,88.89,89.09,2157436,16991,89.479051,DGX
2019-04-17 04:00:00+00:00,89.20,89.400,84.67,85.70,1576775,17327,86.043885,DGX
2019-04-18 04:00:00+00:00,85.78,86.560,84.32,85.75,1486535,13929,85.514119,DGX


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2019-04-12 04:00:00+00:00,162.00,162.5000,159.5000,160.71,2844167,29743,160.783560,CRM
2019-04-15 04:00:00+00:00,161.00,162.0310,159.4900,160.09,2608365,31428,160.412748,CRM
2019-04-16 04:00:00+00:00,160.04,160.2600,157.7501,158.69,3004836,35257,159.336871,CRM
2019-04-17 04:00:00+00:00,159.05,159.0500,155.0700,155.34,4765519,52298,156.091174,CRM
2019-04-18 04:00:00+00:00,154.85,156.2078,153.1700,155.99,4391704,43576,155.169032,CRM


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2019-04-12 04:00:00+00:00,128.88,129.18,127.75,127.92,999817,8284,128.275995,ZBH
2019-04-15 04:00:00+00:00,128.43,128.95,127.64,127.90,857345,7887,127.947027,ZBH
2019-04-16 04:00:00+00:00,128.91,129.54,125.27,125.52,920419,11023,126.506675,ZBH
2019-04-17 04:00:00+00:00,125.84,125.84,119.01,119.69,2338054,24857,120.772755,ZBH
2019-04-18 04:00:00+00:00,119.66,120.96,118.02,120.00,1423091,16918,119.397254,ZBH


In [101]:
filepath = Path('./resources/ohlcv_df_df.csv') 
filepath.parent.mkdir(parents=True, exist_ok=True) 
ohlcv_df.to_csv(filepath)

PermissionError: [Errno 13] Permission denied: 'resources\\ohlcv_df_df.csv'

In [194]:
# Save to DB, be careful here
# ohlcv_df.to_sql('stocks_600', engine)

In [3]:
# Read from DB
Stocks100 = pd.read_sql_table('stocks_100', engine, parse_dates='timestamp')
Stocks200 = pd.read_sql_table('stocks_200', engine, parse_dates='timestamp')
Stocks300 = pd.read_sql_table('stocks_300', engine, parse_dates='timestamp')
Stocks400 = pd.read_sql_table('stocks_400', engine, parse_dates='timestamp')
Stocks500 = pd.read_sql_table('stocks_500', engine, parse_dates='timestamp')
Stocks600 = pd.read_sql_table('stocks_600', engine, parse_dates='timestamp')

In [4]:
Stocks = pd.concat([Stocks100, Stocks200, Stocks300, Stocks400, Stocks500, Stocks600], axis=0)
Stocks['daily return'] = (Stocks['close'] - Stocks['open'])/Stocks['open']
StocksTall = Stocks.drop(columns={'open','high','low','close','volume','trade_count','vwap'})

In [5]:
StocksTall



,timestamp,symbol,daily return
0,2021-04-12 04:00:00,AAL,-0.014194
1,2021-04-13 04:00:00,AAL,0.016216
2,2021-04-14 04:00:00,AAL,-0.025272
3,2021-04-15 04:00:00,AAL,-0.021653
4,2021-04-16 04:00:00,AAL,-0.003169
...,...,...,...
1270,2022-04-06 04:00:00,ZTS,0.017113
1271,2022-04-07 04:00:00,ZTS,0.035199
1272,2022-04-08 04:00:00,ZTS,0.011628
1273,2022-04-11 04:00:00,ZTS,-0.032517


In [12]:
StocksWide = pd.pivot_table(StocksTall, columns='symbol', index = 'timestamp', values='daily return')
StocksWide.reset_index(inplace=True)
StocksWide

symbol,timestamp,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2021-04-12 04:00:00,-0.001293,-0.014194,0.021788,-0.007705,0.003988,0.008408,-0.005014,0.003981,0.005490,...,-0.007863,0.009064,-0.022026,-0.002178,0.003796,0.012348,-0.002134,-0.003978,0.012701,0.020407
1,2021-04-13 04:00:00,0.010410,0.016216,-0.007205,0.015125,0.001759,0.001968,-0.011522,0.016192,-0.004815,...,0.011212,0.013618,0.008143,0.002355,-0.010348,0.008414,0.005851,-0.010332,-0.007684,0.007830
2,2021-04-14 04:00:00,-0.008490,-0.025272,0.001600,-0.022000,-0.003013,0.009064,0.002824,-0.006947,0.000844,...,-0.007537,0.004826,0.017936,0.018744,0.007477,0.005360,-0.003552,-0.005917,0.015529,-0.005670
3,2021-04-15 04:00:00,0.004598,-0.021653,0.006214,0.004781,0.008491,0.005066,0.012594,0.013658,-0.003878,...,-0.014030,0.020566,-0.004716,-0.005371,0.001019,-0.007259,0.018743,0.005353,-0.011160,0.009272
4,2021-04-16 04:00:00,0.003954,-0.003169,0.008574,-0.003308,0.003907,0.001590,-0.026277,-0.000322,0.006312,...,-0.005785,0.001423,-0.010651,-0.000306,-0.005585,-0.004889,-0.004979,0.004952,-0.004079,0.008179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2022-04-06 04:00:00,0.005303,0.008121,0.013428,-0.003195,0.026871,0.017239,-0.012371,0.007899,0.005823,...,-0.017034,0.025154,-0.000717,-0.001241,0.014508,0.013673,-0.004206,0.012861,-0.005312,0.017113
251,2022-04-07 04:00:00,0.022799,-0.009351,0.028212,0.006196,0.033459,0.030292,0.010438,0.031522,0.008201,...,-0.025942,-0.002275,0.014916,0.000208,0.003820,0.018010,0.020342,0.000765,-0.006135,0.035199
252,2022-04-08 04:00:00,-0.006190,-0.016598,0.037742,-0.009665,0.011329,0.011742,-0.002632,-0.001377,-0.010250,...,-0.017813,0.000936,0.016624,0.000625,-0.003106,0.003790,-0.002386,-0.001513,0.003298,0.011628
253,2022-04-11 04:00:00,-0.014713,0.028485,-0.021131,-0.017894,-0.029543,-0.018407,-0.042728,-0.027859,-0.026272,...,-0.000140,-0.007077,-0.023524,0.026032,-0.009245,-0.005213,-0.017041,-0.005064,0.004221,-0.032517


In [13]:
# StocksWide.to_sql('StocksWide', engine, if_exists='replace')

In [8]:
# For 3Y
# Read from DB
Stocks3Y100 = pd.read_sql_table('stocks3Y_100', engine, parse_dates='timestamp')
Stocks3Y200 = pd.read_sql_table('stocks3Y_200', engine, parse_dates='timestamp')
Stocks3Y300 = pd.read_sql_table('stocks3Y_300', engine, parse_dates='timestamp')
Stocks3Y400 = pd.read_sql_table('stocks3Y_400', engine, parse_dates='timestamp')
Stocks3Y500 = pd.read_sql_table('stocks3Y_500', engine, parse_dates='timestamp')
Stocks3Y600 = pd.read_sql_table('stocks3Y_600', engine, parse_dates='timestamp')

In [9]:
Stocks3Y = pd.concat([Stocks3Y100, Stocks3Y200, Stocks3Y300, Stocks3Y400, Stocks3Y500, Stocks3Y600], axis=0)
Stocks3Y['daily return'] = (Stocks3Y['close'] - Stocks3Y['open'])/Stocks3Y['open']
StocksTall3Y = Stocks3Y.drop(columns={'open','high','low','close','volume','trade_count','vwap'})
StocksWide3Y = pd.pivot_table(StocksTall3Y, columns='symbol', index = 'timestamp')

In [10]:
dailyreturns = [ ]
for column in StocksWide3Y.columns.values:
    dailyreturns.append ( StocksWide3Y [ column ].tolist () )

tickers = []
count = 0
for column in StocksWide3Y.columns:
    tickers.append (StocksWide3Y.columns[count][1])
    count += 1

dateTime = StocksWide3Y.index.to_list()
StocksWideFixed3Y = pd.DataFrame()
StocksWideFixed3Y['dateTime'] = dateTime

StockCount = 0
while StockCount < 504:
    StocksWideFixed3Y[tickers[StockCount]] = dailyreturns[StockCount]
    StockCount += 1
StocksWideFixed3Y = StocksWideFixed3Y.copy()

C:\Users\canav\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
